# Generate multiple plots about Kr data

<i>
This notebook takes a kDST file and a correction file,
applies all corrections (including time) and makes all
kind of plots about the data.
</i>

### Notebook configuration

In [ ]:
run_number          = XXX_RUN_NUMBER_XXX
correction_filename = f"$IC_DATA/XYmaps/corrections_run{run_number}.h5"
dst_filename        = f"$IC_DATA/Kr/dst_{run_number}_filtered.root.h5"
plots_output_folder = f"plots/{run_number}/"

XY_pitch            = 5.0

Zrange              =     0, 560
XYrange             =  -200, 200
Rrange              =     0, max(XYrange)
Phirange            = -3.14, 3.14

Erange              =  33.5, 49.5
Qrange              =   200, 1000
QErange             =     0, Qrange[1]/Erange[1]

S1Erange            =     0, 40
S1Wrange            =     0, 500
S1Hrange            =     0, 10

S2Erange            =     0, 15e3
S2Wrange            =     0, 50
S2Hrange            =     0, 10e3
NSirange            =     0, 80

Zrange_DV           =   500, 600
Zrange_LT           =    50, 520
Zrange_XY           =     0, 520

XYnbins             =  50
Znbins              = 100
Rnbins              = 100
Phinbins            = 100
Enbins              = 200
Qnbins              = 200

S1Enbins            = 40
S1Wnbins            = 20
S1Hnbins            = 50

S2Enbins            = 100
S2Wnbins            = 50
S2Hnbins            = 80
NSinbins            = 80

DVnbins             = 50

R_fiducial          = 150
Z_fiducial          = 150

# Plotting style
default_cmap        = "jet"
figure_size         = 16, 12
font_size           = 20
global_linewidth    = 4
subfit_linewidth    = 4
global_linecolor    = "r"
subfit_linecolor    = "m g b y".split()
subfit_linecolor    = "b m slategrey c b y".split()

save_plots          = True
with_titles         = False
plots_format        = "pdf"

### Imports

In [ ]:
import os
import time
import collections
print("Running on ", time.asctime())

import numpy             as np
import matplotlib.pyplot as plt

import invisible_cities.database.load_db        as DB
import invisible_cities.core    .core_functions as coref
import invisible_cities.core    .fit_functions  as fitf
import invisible_cities.reco    .corrections    as corrf
import invisible_cities.reco    .dst_functions  as dstf

from invisible_cities.icaro.hst_functions import hist
from invisible_cities.icaro.hst_functions import hist2d
from invisible_cities.icaro.hst_functions import pdf
from invisible_cities.icaro.hst_functions import display_matrix
from invisible_cities.icaro.hst_functions import resolution
from invisible_cities.icaro.hst_functions import poisson_sigma
from invisible_cities.icaro.hst_functions import gausstext
from invisible_cities.icaro.hst_functions import shift_to_bin_centers
from invisible_cities.icaro.hst_functions import plot_writer
from invisible_cities.icaro.hst_functions import measurement_string

from icaro.core.fit_functions import conditional_labels

### Initialization

In [ ]:
save   = plot_writer(plots_output_folder, plots_format) if save_plots else lambda *x: None
labels = conditional_labels(with_titles)

In [ ]:
%matplotlib inline

plt.rcParams["figure.figsize"         ] = figure_size
plt.rcParams[  "font.size"            ] =   font_size
plt.rcParams["figure.max_open_warning"] = 1000

In [ ]:
Zbins     = np.linspace(*  Zrange   ,   Znbins + 1)
Ebins     = np.linspace(*  Erange   ,   Enbins + 1)
Qbins     = np.linspace(*  Qrange   ,   Qnbins + 1)

S1Ebins   = np.linspace(*S1Erange   , S1Enbins + 1)
S1Wbins   = np.linspace(*S1Wrange   , S1Wnbins + 1)
S1Hbins   = np.linspace(*S1Hrange   , S1Hnbins + 1)

S2Ebins   = np.linspace(*S2Erange   , S2Enbins + 1)
S2Wbins   = np.linspace(*S2Wrange   , S2Wnbins + 1)
S2Hbins   = np.linspace(*S2Hrange   , S2Hnbins + 1)
NSibins   = np.linspace(*NSirange   , NSinbins + 1)

XYbins    = np.linspace(* XYrange   ,  XYnbins + 1)
Rbins     = np.linspace(*  Rrange   ,   Rnbins + 1)
Phibins   = np.linspace(*Phirange   , Phinbins + 1)
DVbins    = np.linspace(*  Zrange_DV,  DVnbins + 1)

dst_filename        = os.path.expandvars(       dst_filename)
correction_filename = os.path.expandvars(correction_filename)

### Read data

In [ ]:
dst = dstf.load_dst(dst_filename, "DST", "Events")
dst = dst[dst.R<max(Rrange)] # soft radial cut to reduce badly reconstructed events
print(len(dst))
dst.head()

In [ ]:
ELT_correction  = dstf.load_lifetime_xy_corrections(correction_filename,
                                                    group = "XYcorrections",
                                                    node  = "Lifetime")

QLT_correction  = dstf.load_lifetime_xy_corrections(correction_filename,
                                                    group = "XYcorrections",
                                                    node  = "QLifetime")

EGEO_correction = dstf.load_xy_corrections(correction_filename,
                                           group         =  "XYcorrections",
                                           node          = f"GeometryE_{XY_pitch:.1f}mm",
                                           norm_strategy =  "index",
                                           norm_opts     = {"index": (40, 40)})

QGEO_correction = dstf.load_xy_corrections(correction_filename,
                                           group         =  "XYcorrections",
                                           node          = f"GeometryQ_{XY_pitch:.1f}mm",
                                           norm_strategy =  "index",
                                           norm_opts     = {"index": (40, 40)})

In [ ]:
evt = dst.event.values
T   =(dst.time .values -
      dst.time.values.min()) / 60
S2n = dst.nS2  .values

S1e = dst.S1e.values
S1w = dst.S1w.values
S1h = dst.S1h.values
S1t = dst.S1t.values

S2e = dst.S2e.values
S2w = dst.S2w.values
S2h = dst.S2h.values
S2t = dst.S2t.values
S2q = dst.S2q.values

X   = dst.X    .values
Y   = dst.Y    .values
Z   = dst.DT   .values
R   = dst.R    .values
Phi = dst.Phi  .values
Nsi = dst.Nsipm.values

E   = S2e * ELT_correction(Z, X, Y).value * EGEO_correction(X, Y).value
Q   = S2q * QLT_correction(Z, X, Y).value * QGEO_correction(X, Y).value

In [ ]:
t, e, eu = fitf.profileX(T, E, 100)
ET_correction = corrf.Correction((t,), e, eu,
                                 norm_strategy =  "const",
                                 norm_opts     = {"value": 41.5})

t, q, qu = fitf.profileX(T, Q, 100)
QT_correction = corrf.Correction((t,), q, qu,
                                 norm_strategy =  "max")

E = E * ET_correction(T).value
Q = Q * QT_correction(T).value

##### Input maps

In [ ]:
LTmap = dstf.load_dst(correction_filename,
                      group = "XYcorrections",
                      node  = "Lifetime")

x   = np.unique(LTmap.x.values)
y   = np.unique(LTmap.y.values)
LT  = LTmap.factor     .values.reshape(x.size, y.size)
LTu = LTmap.uncertainty.values.reshape(x.size, y.size)

*_, cb = display_matrix(x, y, LT,
                        vmin = np.mean(LT) - 2.5 * np.std(LT),
                        vmax = np.mean(LT) + 2.5 * np.std(LT),
                        cmap = default_cmap)
cb.set_label("Lifetime (µs)")
labels("X (mm)", "Y (mm)", "Lifetime map")
save("Lifetime_map")

*_, cb = display_matrix(x, y, LTu,
                        vmin = 1e-3,
                        vmax = np.mean(LTu) + np.std(LTu),
                        cmap = default_cmap)

cb.set_label("Lifetime unceratinty (µs)")
labels("X (mm)", "Y (mm)", "Lifetime uncertainty map")
save("Lifetime_uncertainty_map")

In [ ]:
LTmap = dstf.load_dst(correction_filename,
                      group = "XYcorrections",
                      node  = "QLifetime")

x   = np.unique(LTmap.x.values)
y   = np.unique(LTmap.y.values)
LT  = LTmap.factor     .values.reshape(x.size, y.size)
LTu = LTmap.uncertainty.values.reshape(x.size, y.size)

*_, cb = display_matrix(x, y, LT,
                        vmin = np.mean(LT) - 2.5 * np.std(LT),
                        vmax = np.mean(LT) + 2.5 * np.std(LT),
                        cmap = default_cmap)
cb.set_label("Charge lifetime (µs)")
labels("X (mm)", "Y (mm)", "Charge lifetime map")
save("Charge_lifetime_map")

*_, cb = display_matrix(x, y, LTu,
                        vmin = 1e-3,
                        vmax = np.mean(LTu) + np.std(LTu),
                        cmap = default_cmap)

cb.set_label("Charge lifetime unceratinty (µs)")
labels("X (mm)", "Y (mm)", "Charge lifetime uncertainty map")
save("Charge_lifetime_uncertainty_map")

In [ ]:
GEOmap = dstf.load_dst(correction_filename,
                       group = "XYcorrections",
                       node  = f"GeometryE_{XY_pitch:.1f}mm")

x  = np.unique(GEOmap.x.values)
y  = np.unique(GEOmap.y.values)
f  = GEOmap.factor     .values.reshape(x.size, y.size)
fu = GEOmap.uncertainty.values.reshape(x.size, y.size)

*_, cb = display_matrix(x, y, f,
                        cmap = default_cmap)
cb.set_label("Energy(pes)")
labels("X (mm)", "Y (mm)", "Geometry map")
save("Energy_map")

*_, cb = display_matrix(x, y, fu,
                        vmax = np.mean(fu) + 3 * np.std(fu),
                        cmap = default_cmap)
cb.set_label("Energy unceratinty (pes)")
labels("X (mm)", "Y (mm)", "Geometry uncertainty map")
save("Energy_uncertainty_map")

In [ ]:
GEOmap = dstf.load_dst(correction_filename,
                       group = "XYcorrections",
                       node  = f"GeometryQ_{XY_pitch:.1f}mm")

x  = np.unique(GEOmap.x.values)
y  = np.unique(GEOmap.y.values)
f  = GEOmap.factor     .values.reshape(x.size, y.size)
fu = GEOmap.uncertainty.values.reshape(x.size, y.size)

*_, cb = display_matrix(x, y, f,
                        vmin = Qrange[0],
                        vmax = Qrange[1],
                        cmap = default_cmap)
cb.set_label("Charge(pes)")
labels("X (mm)", "Y (mm)", "Charge map")
save("Charge_map")

*_, cb = display_matrix(x, y, fu,
                        vmax = np.mean(fu) + 3 * np.std(fu),
                        cmap = default_cmap)
cb.set_label("Charge unceratinty (pes)")
labels("X (mm)", "Y (mm)", "Charge uncertainty map")
save("Charge_uncertainty_map")

##### Number of S2s

In [ ]:
hist(S2n, 5, (0, 5))
plt.yscale("log")
labels("Number of S2 signals", "Entries", "Number of S2 signals")
save("NS2")

##### Z distribution

In [ ]:
hist(Z, Zbins)
labels("Drift time ($\mu$s)", "Entries", "Z distribution")
save("Z")

##### Trigger rate

In [ ]:
y, x, _ = hist(T, 100, (0, np.max(T)))
dt      = np.diff(x)[0]

labels( "Time (min)",
       f"Event rate (({dt:.2f} min)$^{{-1}}$)",
        "Trigger rate")
save("EventRate")

##### Raw spectrum

In [ ]:
pdf(S2e, S2Ebins)
labels("S2 energy (pes)", "Entries", "S2 energy")
save("Raw_energy_wide")

hist(S2e, S2Ebins)
labels("S2 energy (pes)", "Entries", "S2 energy")
save("Raw_energy")

##### Drift velocity

In [ ]:
y, x, _  = hist(Z, DVbins)

sigmoid  = lambda x, A, B, C, D: A / (1 + np.exp((x - B) / C)) + D
seed     = np.max(y), np.mean(Zrange_DV), np.diff(Zrange_DV)[0] / 100, np.min(y)
f        = fitf.fit(sigmoid, x, y, seed, sigma=poisson_sigma(y))

z_cathode = DB.DetectorGeo().ZMAX[0]
dv        = z_cathode / f.values[1]
u_dv      =        dv / f.values[1] * f.errors[1]

plt.plot(x, f.fn(x), lw=global_linewidth, c=global_linecolor)
labels("Drift time ($\mu$s)", "Entries", "Z distribution around cathode")

print("Max drift length = {:.3f} +- {:.3f}".format(f.values[1], f.errors[1]))
print("Drift velocity   = {:.5f} +- {:.5f}".format(dv, u_dv))
save("Drift_velocity")

##### S1 distributions

In [ ]:
hist(S1e, S1Ebins)
labels("S1 energy (pes)", "Entries", "S1 energy")
save("S1energy")

hist2d(Z, S1e, (Zbins, S1Ebins))
labels("Drift time ($\mu$s)", "S1 energy (pes)", "S1 energy vs Z")
save("S1energyZ")

In [ ]:
short_drift = coref.in_range(  Z, 0, Z_fiducial) &\
              coref.in_range(S1e, *S1Erange)

x, y, S, u_S = fitf.profileXY(X  [short_drift],
                              Y  [short_drift],
                              S1e[short_drift],
                              XYnbins, XYnbins,
                              XYrange, XYrange)
_, cb = display_matrix(x, y, S)
cb.set_label("S1 energy (pes)")
labels("x (mm)", "y (mm)", "S1 energy vs XY")
save("S1energyXY")

In [ ]:
hist(S1w, S1Wbins)
labels("S1 width (ns)", "Entries", "S1 width")
save("S1width")

hist2d(Z, S1w, (Zbins, S1Wbins))
labels("Drift time ($\mu$s)", "S1 width (ns)", "S1 width vs Z")
save("S1widthZ")

In [ ]:
hist(S1h, S1Hbins)
labels("S1 height (pes)", "Entries", "S1 height")
save("S1height")

hist2d(Z, S1h, (Zbins, S1Hbins))
labels("Drift time ($\mu$s)", "S1 height (pes)", "S1 height vs Z")
save("S1heightZ")

In [ ]:
S1r   = S1h / S1e
_bins = np.linspace(0, np.max(S1r), 50)

hist(S1r, _bins)
labels("Height/Area", "Entries", "S1 peak to area ratio")
save("S1Peak2Area")

hist2d(Z, S1r, (Zbins, _bins))
labels("Drift time ($\mu$s)", "Height/Area", "S1 peak to area ratio vs Z")
save("S1Peak2AreaZ")

##### S2 distributions

In [ ]:
hist(S2e, S2Ebins)
labels("S2 energy (pes)", "Entries", "S2 energy")
save("S2energy")

hist2d(Z, S2e, (Zbins, S2Ebins))
labels("Drift time ($\mu$s)", "S2 energy (pes)", "S2 energy vs Z")
save("S2energyZ")

In [ ]:
hist(S2w, S2Wbins)
labels("S2 width ($\mu$s)", "Entries", "S2 width")
save("S2width")

hist2d(Z, S2w, (Zbins, S2Wbins))
labels("Drift time ($\mu$s)", "S2 width (µs)", "S2 width vs Z")
save("S2widthZ")

In [ ]:
hist(S2h, S2Hbins)
labels("S2 height (pes)", "Entries", "S2 height")
save("S2height")

hist2d(Z, S2h, (Zbins, S2Hbins))
labels("Drift time ($\mu$s)", "S2 height (pes)")
save("S2heightZ")

In [ ]:
S2r   = S2h / S2e
_bins = np.linspace(0, np.max(S2r), 100)


hist(S2r, _bins)
labels("Height/Area", "Entries", "S2 peak to area ratio")
save("S2Peak2Area")

hist2d(Z, S2r, (Zbins, _bins))
labels("Drift time ($\mu$s)", "Height/Area", "S2 peak to area ratio vs Z")
save("S2Peak2AreaZ")

# S2 vs S1

In [ ]:
hist2d(S1e, S2e, (S1Ebins, S2Ebins))
labels("S1 energy (pes)", "S2 energy (pes)", "S2 vs S1")
save("S2S1")

In [ ]:
S12r  = S1e / S2e
_bins = np.linspace(0, np.mean(S12r) + 3 * np.std(S12r), 100)

hist(S12r, _bins)
labels("S1/S2", "Entries", "S1/S2 ratio")
save("S12ratio")

hist2d(Z, S12r, (Zbins, _bins))
labels("Drift time ($\mu$s)", "S1/S2", "S1/S2 ratio vs Z")
save("S12ratioZ")

##### Tracking

In [ ]:
hist(Nsi, NSibins)
labels("# SiPMs", "Entries", "Number of SiPMs touched")
save("Nsipm")

hist2d(Z, Nsi, (Zbins, NSibins))
labels("Drift time ($\mu$s)", "# SiPMs", "Number of SiPMs touched vs Z")
save("NsipmZ")

In [ ]:
hist(X, XYbins)
labels("x (mm)", "Entries", "X distribution")
save("X")

hist(Y, XYbins)
labels("y (mm)", "Entries", "Y distribution")
save("Y")

hist(R, Rbins)
labels("r (mm)", "Entries", "R distribution")
save("R")

hist(Phi, Phibins)
labels("$\phi$ (rad)", "Entries", "$\phi$ distribution")
save("Phi")

hist2d(X, Y, (XYbins, XYbins))
plt.colorbar().set_label("# events")
labels("x (mm)", "y (mm)", "XY distribution")
save("XY")

hist2d(R, Phi, (Rbins, Phibins))
plt.colorbar().set_label("# events")
labels("r (mm)", "$\phi$ (mm)", "R$\phi$ distribution")
save("RPhi")

In [ ]:
hist(S2q, Qbins)
labels("S2 charge (pes)", "Entries", "Charge")
save("Q")

hist2d(Z, S2q, (Zbins, Qbins))
labels("Drift time ($\mu$s)", "S2 charge (pes)", "Charge vs Z")
save("QZ")

##### Energy dependence

In [ ]:
x, y, e, eu = \
fitf.profileXY(X, Y, E, XYnbins, XYnbins, XYrange, XYrange)
_, cb = display_matrix(x, y, e, vmin=np.min(e[e > 0]),
                       cmap = default_cmap)
cb.set_label("Energy (keV)")
labels("x (mm)", "y (mm)", "Energy vs XY")
save("EnergyXY")

r, phi, e, eu = \
fitf.profileXY(R, Phi, E, Rnbins, Phinbins, Rrange, Phirange)
_, cb = display_matrix(r, phi, e, vmin=np.min(e[e > 0]),
                       cmap = default_cmap)
cb.set_label("Energy (keV)")
labels("r (mm)", "$\phi$ (rad)", "Energy vs R$\phi$")
save("EnergyRPhi")

hist2d(Z, E, (Zbins, Ebins))
labels("Drift time (µs)", "SEnergy (keV)")
save("EnergyZ")

hist2d(R, E, (Rbins, Ebins))
labels("R (mm)", "Energy (keV)")
save("EnergyR")

hist2d(Phi, E, (Phibins, Ebins))
labels("$\phi$ (rad)", "Energy (keV)")
save("EnergyPhi")

hist2d(T, E, (np.linspace(0, T.max(), 100), Ebins))
labels("Time (min)", "Energy (keV)", "Energy vs Time")
save("EnergyTime")

### Charge dependence

In [ ]:
x, y, q, qu = \
fitf.profileXY(X, Y, Q, XYnbins, XYnbins, XYrange, XYrange)
_, cb = display_matrix(x, y, q, vmin=np.min(q[q > 0]),
                       cmap = default_cmap)
cb.set_label("Charge (pes)")
labels("x (mm)", "y (mm)", "Charge vs XY")
save("ChargeXY")

r, phi, q, qu = \
fitf.profileXY(R, Phi, Q, Rnbins, Phinbins, Rrange, Phirange)
_, cb = display_matrix(r, phi, q, vmin=np.min(q[q > 0]),
                       cmap = default_cmap)
cb.set_label("Charge (pes)")
labels("r (mm)", "$\phi$ (rad)", "Charge vs R$\phi$")
save("ChargeRPhi")

hist2d(Z, Q, (Zbins, Qbins))
labels("Drift time (µs)", "Charge (keV)")
save("ChargeZ")

hist2d(R, Q, (Rbins, Qbins))
labels("R (mm)", "Charge (keV)")
save("ChargeR")

hist2d(Phi, Q, (Phibins, Qbins))
labels("$\phi$ (rad)", "Charge (keV)")
save("ChargePhi")

hist2d(T, Q, (np.linspace(0, T.max(), 100), Qbins))
labels("Time (min)", "Charge (pes)", "Charge vs Time")
save("ChargeTime")

##### Charge/Energy ratio

In [ ]:
QEratio = Q / E
QEratio = np.where(np.isfinite(QEratio), QEratio, 0)

QEbins = np.linspace(np.mean(QEratio) - 5 * np.std(QEratio),
                     np.mean(QEratio) + 5 * np.std(QEratio), 100)
hist(QEratio, QEbins)
labels("Q/E (pes/keV)", "Entries")
save("QEratio")

hist2d(Z, QEratio, (Zbins, QEbins))
labels("Drift time (µs)", "Q/E (pes/keV)", "Charge/Energy ratio vs Z")
save("QEratioZ")

x, y, qe, qeu = \
fitf.profileXY(X, Y, QEratio, XYnbins, XYnbins, XYrange, XYrange)
_, cb = display_matrix(x, y, qe,
                       cmap = default_cmap)
cb.set_label("Q/E (pes/keV)")
labels("x (mm)", "y (mm)", "Charge/Energy ratio vs XY")
save("QEratioXY")

##### Energy resolution

In [ ]:
xy_data       = collections.namedtuple(      "xy_data", "x y xu yu"       )
reso_fit_data = collections.namedtuple("reso_fit_data", "xy residuals f R")

def measure_resolution(E, bkg=fitf.gauss):
    y, x  = np.histogram(E, Ebins)
    x     = shift_to_bin_centers(x)
    xu    = np.diff(x)[0] * 0.5
    yu    = poisson_sigma(y)

    fun   = lambda x, *args: fitf.gauss(x, *args[:3]) + bkg(x, *args[3:])
    Npeak = np.size(E) * np.diff(Ebins)[0]
    Epeak = np.average( x            , weights=y)
    Speak = np.average((x - Epeak)**2, weights=y)**0.5

    bkg_seed = (Npeak*2, Epeak, Speak*50) if bkg is fitf.gauss else (1, 1, 1)
    sig_seed  = Npeak, Epeak, Speak

    seed = (*sig_seed, *bkg_seed)
    fit_range = (Epeak - 3 * Speak,
                 Epeak + 3 * Speak)
    f    = fitf.fit(fun, x, y, seed,
                    fit_range = fit_range,
                    sigma     = yu,
                    maxfev    = 10000)

    assert not np.any(np.isclose(f.values, seed)), f.values

    R   = resolution(f.values, f.errors, 41.5)
    sel = coref.in_range(x, *fit_range)
    ry  = (f.fn(x) - y) / yu
    return reso_fit_data(xy_data(x, y, xu, yu),
                         xy_data(x[sel], ry[sel], xu, yu[sel]),
                         f, R)

In [ ]:
data = measure_resolution(E, bkg=fitf.gauss)

print("Resolution at 41.5 keV:", measurement_string(*data.R[0]))
print("Resolution at 2458 keV:", measurement_string(*data.R[1]))
print("Chi2                  :", data.f.chi2)

x = data.xy.x
plt.errorbar    (x, data.xy.y, data.xy.yu, data.xy.xu, "p", c="k")
plt.plot        (x, data.f.fn   (x)                   , lw=global_linewidth, color=global_linecolor   )
plt.plot        (x, fitf.gauss(x, *data.f.values[ :3]), lw=subfit_linewidth, color=subfit_linecolor[0])
plt.fill_between(x, fitf.gauss(x, *data.f.values[ :3]),    0,     alpha=0.3, color=subfit_linecolor[0])
plt.plot        (x, fitf.gauss(x, *data.f.values[3: ]), lw=subfit_linewidth, color=subfit_linecolor[1])
plt.fill_between(x, fitf.gauss(x, *data.f.values[3: ]),    0,     alpha=0.3, color=subfit_linecolor[1])
plt.text(plt.xlim()[0] + np.diff(plt.xlim()) * 0.05,
         max(data.xy.y) / 1.5,
         gausstext(data.f.values, data.f.errors, 41.5))


plt.ylim(0)
labels("E (keV)", "Entries", "Energy spectrum")
save("EnergySpectrum")

In [ ]:
frame_data = plt.gcf().add_axes((.1, .3,
                                 .8, .6))
plt.errorbar    (x, data.xy.y, data.xy.yu, data.xy.xu, "p", c="k")
plt.plot        (x, data.f.fn (x)                     , lw=global_linewidth, color=global_linecolor   )
plt.plot        (x, fitf.gauss(x, *data.f.values[ :3]), lw=subfit_linewidth, color=subfit_linecolor[0])
plt.fill_between(x, fitf.gauss(x, *data.f.values[ :3]),    0,     alpha=0.3, color=subfit_linecolor[0])
plt.plot        (x, fitf.gauss(x, *data.f.values[3: ]), lw=subfit_linewidth, color=subfit_linecolor[1])
plt.fill_between(x, fitf.gauss(x, *data.f.values[3: ]),    0,     alpha=0.3, color=subfit_linecolor[1])
frame_data.set_xticklabels([])
plt.ylabel("Entries")
plt.ylim(0)
lims = plt.xlim()

frame_res = plt.gcf().add_axes((.1, .1,
                                .8, .2))
plt.errorbar(data.residuals.x, data.residuals.y, 1, data.residuals.xu, fmt="p", c="k")
plt.plot    (lims, [0,0], "g--")
plt.xlim(*lims)
labels("E (keV)", "Standarized residual")
save("EnergySpectrumCool")

##### Energy resolution best case

In [ ]:
sel_r = coref.in_range(R, 0, R_fiducial)
data  = measure_resolution(E[sel_r], bkg=fitf.gauss)

print("Resolution at 41.5 keV:", measurement_string(*data.R[0]))
print("Resolution at 2458 keV:", measurement_string(*data.R[1]))
print("Chi2                  :", data.f.chi2)

x = data.xy.x
plt.errorbar    (x, data.xy.y, data.xy.yu, data.xy.xu, "p", c="k")
plt.plot        (x, data.f.fn (x)                     , lw=global_linewidth, color=global_linecolor   )
plt.plot        (x, fitf.gauss(x, *data.f.values[ :3]), lw=subfit_linewidth, color=subfit_linecolor[0])
plt.fill_between(x, fitf.gauss(x, *data.f.values[ :3]),    0,     alpha=0.3, color=subfit_linecolor[0])
plt.plot        (x, fitf.gauss(x, *data.f.values[3: ]), lw=subfit_linewidth, color=subfit_linecolor[1])
plt.fill_between(x, fitf.gauss(x, *data.f.values[3: ]),    0,     alpha=0.3, color=subfit_linecolor[1])

plt.text(plt.xlim()[0] + np.diff(plt.xlim()) * 0.05,
         max(data.xy.y) / 1.5,
         gausstext(data.f.values, data.f.errors, 41.5))


plt.ylim(0)
labels("E (keV)", "Entries", f"Energy spectrum for R < {R_fiducial:.0f} mm")
save("EnergySpectrumFiducialR")

In [ ]:
sel_z = coref.in_range(Z, 0, Z_fiducial)
data  = measure_resolution(E[sel_z], bkg=fitf.gauss)

print("Resolution at 41.5 keV:", measurement_string(*data.R[0]))
print("Resolution at 2458 keV:", measurement_string(*data.R[1]))
print("Chi2                  :", data.f.chi2)

x = data.xy.x
plt.errorbar    (x, data.xy.y, data.xy.yu, data.xy.xu, "p", c="k")
plt.plot        (x, data.f.fn (x)                     , lw=global_linewidth, color=global_linecolor   )
plt.plot        (x, fitf.gauss(x, *data.f.values[ :3]), lw=subfit_linewidth, color=subfit_linecolor[0])
plt.fill_between(x, fitf.gauss(x, *data.f.values[ :3]),    0,     alpha=0.3, color=subfit_linecolor[0])
plt.plot        (x, fitf.gauss(x, *data.f.values[3: ]), lw=subfit_linewidth, color=subfit_linecolor[1])
plt.fill_between(x, fitf.gauss(x, *data.f.values[3: ]),    0,     alpha=0.3, color=subfit_linecolor[1])
plt.text(plt.xlim()[0] + np.diff(plt.xlim()) * 0.05,
         max(data.xy.y) / 1.5,
         gausstext(data.f.values, data.f.errors, 41.5))


plt.ylim(0)
labels("E (keV)", "Entries", f"Energy spectrum for Z < {Z_fiducial:.0f} µs")
save("EnergySpectrumFiducialZ")

In [ ]:
sel_r = coref.in_range(R, 0, R_fiducial)
sel_z = coref.in_range(Z, 0, Z_fiducial)
data  = measure_resolution(E[sel_r & sel_z], bkg=fitf.polynom)

print("Resolution at 41.5 keV:", measurement_string(*data.R[0]))
print("Resolution at 2458 keV:", measurement_string(*data.R[1]))
print("Chi2                  :", data.f.chi2)

x = data.xy.x
plt.errorbar    (x, data.xy.y, data.xy.yu, data.xy.xu, "p", c="k")
plt.plot        (x, data.f.fn   (x)                     , lw=global_linewidth, color=global_linecolor   )
plt.plot        (x, fitf.gauss  (x, *data.f.values[ :3]), lw=subfit_linewidth, color=subfit_linecolor[0])
plt.fill_between(x, fitf.gauss  (x, *data.f.values[ :3]),    0,     alpha=0.3, color=subfit_linecolor[0])
plt.plot        (x, fitf.polynom(x, *data.f.values[3: ]), lw=subfit_linewidth, color=subfit_linecolor[1])
plt.fill_between(x, fitf.polynom(x, *data.f.values[3: ]),    0,     alpha=0.3, color=subfit_linecolor[1])
plt.text(plt.xlim()[0] + np.diff(plt.xlim()) * 0.05,
         max(data.xy.y) / 1.5,
         gausstext(data.f.values, data.f.errors, 41.5))


plt.ylim(0)
labels("E (keV)", "Entries", f"Energy spectrum for R < {R_fiducial:.0f} mm, Z < {Z_fiducial:.0f} µs")
save("EnergySpectrumFiducialRZ")

##### Energy resolution dependence

In [ ]:
rbins = np.linspace(*Rrange, 17)
nbins = len(rbins) - 1
r     = shift_to_bin_centers(rbins)
dr    = 0.5 * np.diff       (rbins)

valid         = np.zeros(nbins, dtype=bool     )
Chi2          = np.zeros(nbins, dtype=np.double)
Resolution    = np.zeros(nbins, dtype=np.double)
Resolutionu   = np.zeros(nbins, dtype=np.double)
ResolutionBB  = np.zeros(nbins, dtype=np.double)
ResolutionBBu = np.zeros(nbins, dtype=np.double)

sel_z = coref.in_range(Z, 0, Z_fiducial)
for i in range(nbins):
    plt.subplot(4, 4, i+1)
    try:
        rmin, rmax = rbins[i:i+2]
        sel_r = coref.in_range(R, rmin, rmax)
        sel   = sel_r & sel_z
        if np.count_nonzero(sel) < 100: continue

        data  = measure_resolution(E[sel], bkg=fitf.polynom)

        print(f"R in [{rmin:.1f}, {rmax:.1f}) mm")
        print( "| Resolution at 2458 keV:", measurement_string(*data.R[1]))
        print( "| Chi2                  :", data.f.chi2)

        x = data.xy.x
        plt.errorbar(x, data.xy.y, data.xy.yu, data.xy.xu, "p", c="k")
        plt.plot    (x, data.f.fn   (x)                     , lw=global_linewidth, c=global_linecolor)
        plt.plot    (x, fitf.gauss  (x, *data.f.values[ :3]), lw=subfit_linewidth, c=subfit_linecolor[0])
        plt.plot    (x, fitf.polynom(x, *data.f.values[3: ]), lw=subfit_linewidth, c=subfit_linecolor[1])

        plt.ylim(0)
        labels("E (keV)", "Entries", f"R $\in$ [{rmin:.1f}, {rmax:.1f}) mm")

        Resolution  [i], Resolutionu  [i] = data.R[0]
        ResolutionBB[i], ResolutionBBu[i] = data.R[1]
        Chi2        [i]                   = data.f.chi2
        valid       [i]                   = True
    except:
        pass

plt.tight_layout()
save("ResolutionRslices")

In [ ]:
plt.errorbar(r[valid], Resolution[valid], Resolutionu[valid], dr[valid], "kp")
labels("R (mm)", "Resolution", "Resolution vs R")

new_lims  = [lim * (41.5 / 2458)**0.5 for lim in plt.ylim()]
new_ticks = np.linspace(*new_lims, 7)
str_ticks = ["{:.2f}".format(i) for i in new_ticks]
ax2       = plt.gca().twinx()

ax2.set_ylim       (*new_lims )
ax2.set_yticks     ( new_ticks)
ax2.set_yticklabels( str_ticks)
ax2.set_ylabel("Resolution @ Qbb", rotation=-90, labelpad=20);
save("ResolutionR")

In [ ]:
zbins = np.linspace(*Zrange, 13)
nbins = len(zbins) - 1
z     = shift_to_bin_centers(zbins)
dz    = 0.5 * np.diff       (zbins)

valid         = np.zeros(nbins, dtype=bool     )
Chi2          = np.zeros(nbins, dtype=np.double)
Resolution    = np.zeros(nbins, dtype=np.double)
Resolutionu   = np.zeros(nbins, dtype=np.double)
ResolutionBB  = np.zeros(nbins, dtype=np.double)
ResolutionBBu = np.zeros(nbins, dtype=np.double)

sel_r = coref.in_range(R, 0, R_fiducial)
for i in range(nbins):
    plt.subplot(3, 4, i+1)
    try:
        zmin, zmax = zbins[i:i+2]
        sel_z = coref.in_range(Z, zmin, zmax)
        sel   = sel_r & sel_z
        if np.count_nonzero(sel) < 100: continue

        data  = measure_resolution(E[sel], bkg=fitf.polynom)

        print(f"Z in [{zmin:.1f}, {zmax:.1f}) mm")
        print( "| Resolution at 2458 keV:", measurement_string(*data.R[1]))
        print( "| Chi2                  :", data.f.chi2)

        x = data.xy.x
        plt.errorbar(x, data.xy.y, data.xy.yu, data.xy.xu, "p", c="k")
        plt.plot    (x, data.f.fn   (x)                     , lw=global_linewidth, c=global_linecolor)
        plt.plot    (x, fitf.gauss  (x, *data.f.values[ :3]), lw=subfit_linewidth, c=subfit_linecolor[0])
        plt.plot    (x, fitf.polynom(x, *data.f.values[3: ]), lw=subfit_linewidth, c=subfit_linecolor[1])

        plt.ylim(0)
        labels("E (keV)", "Entries", f"Z $\in$ [{zmin:.1f}, {zmax:.1f}) mm")

        Resolution  [i], Resolutionu  [i] = data.R[0]
        ResolutionBB[i], ResolutionBBu[i] = data.R[1]
        Chi2        [i]                   = data.f.chi2
        valid       [i]                   = True
    except:
        pass

plt.tight_layout()
save("ResolutionZslices")

In [ ]:
plt.errorbar(z[valid], Resolution[valid], Resolutionu[valid], dz[valid], "kp")
labels("Drift time (µs)", "Resolution", "Resolution vs Z")

new_lims  = [lim * (41.5 / 2458)**0.5 for lim in plt.ylim()]
new_ticks = np.linspace(*new_lims, 7)
str_ticks = ["{:.2f}".format(i) for i in new_ticks]
ax2       = plt.gca().twinx()

ax2.set_ylim       (*new_lims )
ax2.set_yticks     ( new_ticks)
ax2.set_yticklabels( str_ticks)
ax2.set_ylabel("Resolution @ Qbb", rotation=-90, labelpad=20);
save("ResolutionZ")

In [ ]:
xybins = np.linspace(*XYrange, 16)
nbins  = len(xybins) - 1
xy     = shift_to_bin_centers(xybins)
dxy    = 0.5 * np.diff       (xybins)

valid         = np.zeros((nbins, nbins), dtype=bool     )
Chi2          = np.zeros((nbins, nbins), dtype=np.double)
Resolution    = np.zeros((nbins, nbins), dtype=np.double)
Resolutionu   = np.zeros((nbins, nbins), dtype=np.double)
ResolutionBB  = np.zeros((nbins, nbins), dtype=np.double)
ResolutionBBu = np.zeros((nbins, nbins), dtype=np.double)

plt.figure(figsize=(30, 30))

sel_z = coref.in_range(Z, 0, Z_fiducial)
for i in range(nbins):
    for j in range(nbins):
        plt.subplot(nbins, nbins, i * nbins + j + 1)

        xmin, xmax = xybins[i : i + 2]
        ymin, ymax = xybins[j : j + 2]
        sel_x = coref.in_range(X, xmin, xmax)
        sel_y = coref.in_range(Y, ymin, ymax)

        try:
            data  = measure_resolution(E[sel_x & sel_y & sel_z], bkg=fitf.polynom)

            x = data.xy.x
            plt.errorbar(x, data.xy.y, data.xy.yu, data.xy.xu, "p", c="k")
            plt.plot    (x, data.f.fn   (x)                     , lw=global_linewidth, c=global_linecolor)
            plt.plot    (x, fitf.gauss  (x, *data.f.values[ :3]), lw=subfit_linewidth, c=subfit_linecolor[0])
            plt.plot    (x, fitf.polynom(x, *data.f.values[3: ]), lw=subfit_linewidth, c=subfit_linecolor[1])

            plt.ylim(0)
            labels("E (keV)", "Entries", f"Z $\in$ [{zmin:.1f}, {zmax:.1f}) mm")

            Resolution  [i, j], Resolutionu  [i, j] = data.R[0]
            ResolutionBB[i, j], ResolutionBBu[i, j] = data.R[1]
            Chi2        [i, j]                      = data.f.chi2
            valid       [i, j]                      = True
        except:
            pass

plt.tight_layout()
save("ResolutionXYbins")

In [ ]:
*_, cb = display_matrix(xy, xy, ResolutionBB, vmin = 0, vmax = 1,
                        cmap = default_cmap)
cb.set_label("Resolution @ Qbb")
labels("X (mm)", "Y (mm)", "Resolution vs XY")
save("ResolutionXY")